<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>


# Deep Learning Basics with PyTorch

**Dr. Yves J. Hilpisch with GPT-5**


# Chapter 11 — Deeper Architectures
Convolutions, pooling, and a tiny CNN.

## Overview

This notebook provides a concise, hands-on walkthrough of Deep Learning Basics with PyTorch.
Use it as a companion to the chapter: run each cell, read the short notes,
and try small variations to build intuition.

Tips:
- Run cells top to bottom; restart kernel if state gets confusing.
- Prefer small, fast experiments; iterate quickly and observe outputs.
- Keep an eye on shapes, dtypes, and devices when using PyTorch.


In [ ]:
# !pip -q install torch numpy matplotlib  # install dependencies if needed
import matplotlib.pyplot as plt  # plotting utilities
import numpy as np  # numerical helpers
import torch  # tensor library
import torch.nn.functional as F  # functional conv helpers
from IPython import get_ipython  # notebook runtime access
plt.style.use("seaborn-v0_8")  # consistent styling
get_ipython().run_line_magic("config", "InlineBackend.figure_format = 'retina'")  # crisp notebook figures

## Convolution (toy example)

Build a tiny synthetic example: a white square inside a dark canvas filtered with a cross-shaped kernel.
The output highlights edges because the kernel sums neighbours and subtracts the centre.

In [ ]:
x = torch.zeros(1, 1, 6, 6)  # blank canvas
x[:, :, 2:4, 2:4] = 1.0  # add bright 2x2 square in the centre
kernel = torch.tensor([[0.0, 1.0, 0.0], [1.0, -4.0, 1.0], [0.0, 1.0, 0.0]]).view(1, 1, 3, 3)  # laplacian-style filter
y = F.conv2d(x, kernel, stride=1, padding=0)  # valid convolution
print("input shape:", tuple(x.shape))  # confirm input shape
print("output shape:", tuple(y.shape))  # confirm output shape
print("output map:\n", y.squeeze())  # display response values


## Padding vs stride (shapes)

Demonstrate how padding preserves spatial size while stride downsamples feature maps.

In [ ]:
sample = torch.randn(1, 1, 5, 5)  # random 5x5 input
kernel = torch.ones(1, 1, 3, 3)  # 3x3 averaging kernel
no_pad = F.conv2d(sample, kernel, stride=1, padding=0)  # valid conv
same_pad = F.conv2d(sample, kernel, stride=1, padding=1)  # keep size
stride_two = F.conv2d(sample, kernel, stride=2, padding=0)  # stride-2 conv
print("no padding:", tuple(no_pad.shape[-2:]))  # show spatial dims
print("padding=1:", tuple(same_pad.shape[-2:]))  # show same-size dims
print("stride=2:", tuple(stride_two.shape[-2:]))  # show downsampled dims

## Tiny CNN (shape tracing)

Stack two conv/pool blocks and confirm intermediate shapes along the forward pass.

In [ ]:
from torch import nn  # neural network modules
class TinyCNN(nn.Module):  # simple two-block CNN
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(  # feature extractor
            nn.Conv2d(1, 8, 3, padding=1),  # conv block 1 keeps resolution
            nn.ReLU(),  # nonlinearity
            nn.MaxPool2d(2),  # downsample by 2
            nn.Conv2d(8, 16, 3, padding=1),  # conv block 2
            nn.ReLU(),  # nonlinearity
            nn.MaxPool2d(2),  # downsample by 2 again
        )
        self.classifier = nn.Sequential(  # linear head
            nn.Flatten(),  # flatten to (B, features)
            nn.Linear(16 * 7 * 7, 10),  # map to logits
        )
    def forward(self, x):  # forward propagation
        x = self.features(x)  # apply conv blocks
        return self.classifier(x)  # project to class scores
model = TinyCNN()  # instantiate network
sample = torch.randn(4, 1, 28, 28)  # mini-batch of images
feats = model.features(sample)  # extract feature maps
logits = model(sample)  # run full network
print("input:", tuple(sample.shape))  # show input shape
print("features:", tuple(feats.shape))  # show feature map shape
print("logits:", tuple(logits.shape))  # show classifier output


## Exercises

1. Compute output shapes layer-by-layer for a CNN; verify with a forward pass.
2. Visualize feature maps for one input and describe qualitative differences.


<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>
